In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [3]:
data = pd.read_pickle('../create_SL_data/data_6months_ua.pkl')

data

,patient_number,cluster_0_size-diff%_pre_to_3h,cluster_1_size-diff%_pre_to_3h,cluster_2_size-diff%_pre_to_3h,cluster_3_size-diff%_pre_to_3h,cluster_4_size-diff%_pre_to_3h,cluster_5_size-diff%_pre_to_3h,cluster_6_size-diff%_pre_to_3h,cluster_7_size-diff%_pre_to_3h,cluster_8_size-diff%_pre_to_3h,...,q90_difference_pre_7d_cluster_19_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_20_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_21_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_22_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_23_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_24_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_25_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_26_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_27_158Gd_pSTAT3_Y705,CHR
0,01,112.311228,4.657895,193.583600,17.357513,131.736527,41.726700,-9.016142,581.484794,341.481481,...,-0.036379,0.538215,0.244545,0.173798,0.520689,0.000000,0.445879,0.528887,-0.634862,1
1,02,-39.227196,-35.480393,-44.449358,-42.916013,-23.653129,-27.315147,-71.392762,-4.200082,-66.785333,...,0.409531,0.471072,0.520922,0.195533,0.814652,0.000000,0.649210,0.523819,-0.824650,1
2,03,107.606356,226.932177,32.365967,344.897484,21.987033,155.866808,96.242718,139.396945,20.366891,...,0.033095,-0.010136,0.182196,0.013442,0.234621,0.450780,0.509549,0.256670,-2.014077,1
3,04,-25.682913,124.070561,135.671642,44.396807,-1.134931,83.993576,2.053080,36.713996,-25.974026,...,0.197905,-0.030211,0.115321,0.107060,0.578006,0.125471,0.312356,0.236907,-1.469164,1
4,05,201.201479,43.747082,122.874251,43.729788,-36.776933,182.597569,-3.854390,119.651741,6.158358,...,0.528722,0.116360,0.030627,0.588062,0.544317,0.414612,0.000000,0.000000,-0.530531,1
5,06,1.924515,15.800274,30.471903,-24.722838,-0.323767,83.695906,-34.709775,-28.399925,50.686604,...,0.228519,-0.704528,0.353815,0.126909,0.615345,0.805617,0.184870,0.000000,-1.011029,0
6,07,-8.081530,-18.147677,66.870229,-41.526781,119.244513,-4.668262,-14.515928,-0.029555,125.590251,...,0.206264,0.310430,0.432708,0.101798,0.122739,-0.723297,0.275844,-0.836546,-0.386960,1
7,08,-31.193715,-35.195035,25.087948,-39.439962,-17.761805,-27.302254,-34.826649,-12.109273,-26.896508,...,0.085245,-0.113622,0.375774,0.155500,-0.032117,0.751987,0.243146,0.494612,-1.508415,1
8,09,12.781223,-20.827075,54.025045,-10.660076,100.369004,-8.090675,-24.769019,-14.171028,209.620596,...,0.838215,0.285612,0.294480,0.515315,0.613103,1.708393,-0.129841,0.539467,-1.008428,1
9,11,-20.563531,27.474541,24.549279,-6.790533,175.652174,2.324571,6.422837,-6.846860,289.677419,...,0.683557,-0.295606,0.232018,0.329252,0.860805,0.000000,-0.124076,-0.068443,-0.809707,1


In [4]:
data['patient_number'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '11', '12',
       '13', '14', '15', '16', '17', '18', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '32', '33', '34', '35', '36', '37',
       '38'], dtype=object)

In [5]:
X = data.drop(['CHR', 'patient_number'], axis=1)  
y = data['CHR']                                  

patient_numbers = data['patient_number']

In [17]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score

models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {'n_estimators': [50, 100, 200, 300, 500], 'max_depth': [3, 5, 10, 50], 'max_features' : ['sqrt', None]}

    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
    },
    'LogisticRegression': {
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {'C': [0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}
    },
    'LDA': {
        'model': LinearDiscriminantAnalysis(),
        'params': {'solver': ['svd', 'lsqr']}
    }
}

feature_importances = {name: pd.DataFrame(np.zeros((X.shape[0], X.shape[1])), columns=X.columns) for name in models if name == 'RandomForest'}
scores = {name: {'f1_scores': [], 'accuracy_scores': [], 'best_params': []} for name in models}

outer_cv = LeaveOneOut()
inner_cv = LeaveOneOut()

for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    for name, config in models.items():
        grid_search = GridSearchCV(config['model'], config['params'], cv=inner_cv, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        
        y_pred = best_model.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)

        scores[name]['f1_scores'].append(f1)
        scores[name]['accuracy_scores'].append(accuracy)
        scores[name]['best_params'].append(grid_search.best_params_)

        if name == 'RandomForest':
            feature_importances[name].iloc[fold_idx, :] = best_model.feature_importances_
        
    print('fold', fold_idx)

results = {}
for name in models:
    average_f1 = np.mean(scores[name]['f1_scores'])
    average_accuracy = np.mean(scores[name]['accuracy_scores'])
    results[name] = {'Average F1 Score': average_f1, 'Average Accuracy': average_accuracy, 'Best Parameters': scores[name]['best_params'][-1]}

    if name == 'RandomForest':
        mean_fi = feature_importances[name].mean(axis=0).sort_values(ascending=False)
        results[name]['Feature Importances'] = mean_fi

for name, result in results.items():
    print(f"{name} Results:")
    for key, value in result.items():
        print(f"{key}: {value}")


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


fold 0


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


fold 1


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


...

fold 31


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


fold 32


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


fold 33
RandomForest Results:
Average F1 Score: 0.9411764705882353
Average Accuracy: 0.9411764705882353
Best Parameters: {'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 50}
Feature Importances: 90th_172Yb_pS6 S235S236_3h_cluster_19                 0.036335
90th_144Nd_pTyr_3h_cluster_18                         0.032709
q90_difference_pre_3h_cluster_25_151Eu_pSTAT3_S727    0.028702
90th_159Tb_pMAPKAPK T334_3h_cluster_11                0.027596
90th_172Yb_pS6 S235S236_3h_cluster_7                  0.025686
                                                        ...   
90th_165Ho_pCREB S133_3h_cluster_27                   0.000000
90th_165Ho_pCREB S133_3h_cluster_26                   0.000000
90th_165Ho_pCREB S133_3h_cluster_25                   0.000000
90th_165Ho_pCREB S133_3h_cluster_24                   0.000000
q90_difference_pre_7d_cluster_27_158Gd_pSTAT3_Y705    0.000000
Length: 2156, dtype: float64
KNN Results:
Average F1 Score: 0.9411764705882353
Average Accuracy: 0.94117

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [7]:
model1 = RandomForestClassifier(n_estimators=50, max_depth=3, max_features='sqrt', random_state=42)

model1.fit(X, y)


RandomForestClassifier(max_depth=3, n_estimators=50, random_state=42)

In [8]:
model1.predict(X)

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [10]:
pd.set_option('display.max_colwidth', None)

feature_importances = model1.feature_importances_
features_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})
features_df = features_df.sort_values(by='Importance', ascending=False)

features_df.head(10
                 )

,Feature,Importance
2123,q90_difference_pre_7d_cluster_23_143Nd_pCRKL_Y207,0.044444
160,90th_150Nd_pSTAT5 Y694_pre_cluster_20,0.032997
1260,90th_173Yb_STAT3tot_7d_cluster_0,0.030093
1727,q90_difference_pre_3h_cluster_19_158Gd_pSTAT3_Y705,0.028315
480,90th_142Nd_cCaspase3_3h_cluster_4,0.025275
1425,q90_difference_pre_3h_cluster_25_151Eu_pSTAT3_S727,0.022222
831,90th_172Yb_pS6 S235S236_3h_cluster_19,0.022222
610,90th_151Eu_pSTAT3 S727_3h_cluster_22,0.022222
64,90th_142Nd_cCaspase3_pre_cluster_8,0.022222
622,90th_153Eu_pSTAT1 Y701_3h_cluster_6,0.022222


## Testing

In [55]:
data_pool2 = pd.read_pickle('../create_SL_data_pool2/data_6months_ua_pool2.pkl')

In [56]:
X_test = data_pool2.drop(['CHR', 'patient_number'], axis=1)  
y_test = data_pool2['CHR']                                  

y_test

0    1
1    0
2    1
3    0
4    1
5    1
6    0
7    1
8    1
Name: CHR, dtype: int64

In [57]:
model1.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [70]:
data_pool2

patient_numbers_pool2 = data_pool2['patient_number'].astype(int)
patient_numbers_pool2

0    39
1    40
2    41
3    42
4    44
5    45
6    54
7    56
8    57
Name: patient_number, dtype: int64

In [72]:
patient_numbers_pool2 = data_pool2['patient_number'].astype(int)
response = pd.read_csv('../response/responses_all.csv')
response = response[response['patient_number'].isin(patient_numbers_pool2)]

response

,patient_number,patient_id,batch,month,BCRABL
237,39,4810_00009,5,3,0.070
238,39,4810_00009,5,6,0.020
239,39,4810_00009,5,9,0.020
240,39,4810_00009,5,12,0.007
241,39,4810_00009,5,15,0.009
...,...,...,...,...,...
295,57,4306_00004,6,9,0.025
296,57,4306_00004,6,12,NaN
297,57,4306_00004,6,15,NaN
298,57,4306_00004,6,17,0.007


In [80]:
import plotly.express as px

response['BCRABL'] = response['BCRABL'].replace(0, 0.001)

fig = px.line(response, x='month', y='BCRABL', color='patient_number', log_y=True,
              title='BCR-ABL% over Time by Patient', markers=True)

fig.update_layout(
    yaxis_title='BCR-ABL%',
    xaxis_title='Test Time (Months)',
    yaxis_type='log'
)

fig.update_traces(connectgaps=True)


fig.add_hline(y=0.01)

fig.add_hline(y=1, line_dash="dash")

fig.show()


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

